In [18]:
from rdflib import Graph
from rdflib.namespace import RDF, RDFS, SKOS
import rdflib
import os
from SPARQLWrapper import SPARQLWrapper, JSON
import pandas as pd
sparql = SPARQLWrapper("http://dbpedia.org/sparql")
sparql.setReturnFormat(JSON)
sparql.setTimeout(20)

schema = rdflib.Namespace("http://schema.org/")
owl = rdflib.Namespace("http://www.w3.org/2002/07/owl#")

In [11]:
directory = r"../../OpenCS/ontology/core/00/"
g = Graph()
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f):
        new_node = Graph()
        new_node.parse(f)
        g += new_node
        

In [12]:
g.triples((None, RDF.type, schema.Place))

<generator object Graph.triples at 0x7fe2ca36d360>

In [20]:
names = [
    "(dbr:" + o.split("/").pop() + ")"
    for s, p, o in g.triples((None, SKOS.closeMatch, None))
]

In [21]:
names

['(dbr:Computer_science)',
 '(dbr:Algorithm)',
 '(dbr:Embedded_system)',
 '(dbr:Reconfigurability)',
 '(dbr:REEM)',
 '(dbr:Crt0)',
 '(dbr:Knowledge_management)',
 '(dbr:Systems_simulation)',
 '(dbr:Cellular_automaton)',
 '(dbr:Digital_filter)',
 '(dbr:Interrogative)',
 '(dbr:Next-generation_network)',
 '(dbr:Logical_consequence)',
 '(dbr:Bookmarking)',
 '(dbr:Global_variable)',
 '(dbr:Functional_capacity_evaluation)',
 '(dbr:Data_mining)',
 '(dbr:Instance_selection)',
 '(dbr:Java_Interface_Definition_Language)',
 '(dbr:Williot_diagram)',
 '(dbr:ClinicalKey)',
 '(dbr:Pro*C)',
 '(dbr:ThoughtTreasure)',
 '(dbr:Oracle_BI_server)',
 '(dbr:Reproduction_speed)',
 '(dbr:Greedy_source)',
 '(dbr:Computer_engineering)',
 '(dbr:Continuous_test-driven_development)',
 '(dbr:Computer_architecture)',
 '(dbr:Limiter)',
 '(dbr:Broadcast_engineering)',
 '(dbr:Abstract_machine)',
 '(dbr:ILLIAC)',
 '(dbr:Crosswind)',
 '(dbr:Timestamp)',
 '(dbr:Speech_recognition)',
 '(dbr:Geometric_primitive)',
 '(dbr:Meth

In [33]:
query

'\n\nPREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>\nPREFIX dbr: <http://dbpedia.org/resource/>\n\nSELECT DISTINCT\n\t ?uri ?label \nWHERE {\n    VALUES (?uri) {(dbr:Computer_science)\n(dbr:Algorithm)\n(dbr:Embedded_system)\n(dbr:Reconfigurability)\n(dbr:REEM)\n(dbr:Crt0)\n(dbr:Knowledge_management)\n(dbr:Systems_simulation)\n(dbr:Cellular_automaton)\n(dbr:Digital_filter)}\n    ?uri rdfs:label ?label\n}\nLIMIT 1000\n'

In [32]:
bsl = "\n"
#needs to be divied into chunks due to limitations of the rdfslib query
query = f"""

PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX dbr: <http://dbpedia.org/resource/>

SELECT DISTINCT
	 ?uri ?label 
WHERE {{
    VALUES (?uri) {{{bsl.join(names[0:10])}}}
    ?uri rdfs:label ?label
}}
LIMIT 1000
"""
sparql.setQuery(query)

try:
    ret = sparql.queryAndConvert()

    for r in ret["results"]["bindings"]:
        print(r)
except Exception as e:
    print(e)

{'uri': {'type': 'uri', 'value': 'http://dbpedia.org/resource/Algorithm'}, 'label': {'type': 'literal', 'xml:lang': 'en', 'value': 'Algorithm'}}
{'uri': {'type': 'uri', 'value': 'http://dbpedia.org/resource/Algorithm'}, 'label': {'type': 'literal', 'xml:lang': 'ar', 'value': 'خوارزمية'}}
{'uri': {'type': 'uri', 'value': 'http://dbpedia.org/resource/Algorithm'}, 'label': {'type': 'literal', 'xml:lang': 'ca', 'value': 'Algorisme'}}
{'uri': {'type': 'uri', 'value': 'http://dbpedia.org/resource/Algorithm'}, 'label': {'type': 'literal', 'xml:lang': 'cs', 'value': 'Algoritmus'}}
{'uri': {'type': 'uri', 'value': 'http://dbpedia.org/resource/Algorithm'}, 'label': {'type': 'literal', 'xml:lang': 'de', 'value': 'Algorithmus'}}
{'uri': {'type': 'uri', 'value': 'http://dbpedia.org/resource/Algorithm'}, 'label': {'type': 'literal', 'xml:lang': 'el', 'value': 'Αλγόριθμος'}}
{'uri': {'type': 'uri', 'value': 'http://dbpedia.org/resource/Algorithm'}, 'label': {'type': 'literal', 'xml:lang': 'eo', 'valu